In [ ]:
!pip install py7zr 
!pip install transformers
!pip install datasets
!pip install rouge-score
!pip install nltk
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 65 kB 4.1 MB/s 
     |████████████████████████████████| 357 kB 36.8 MB/s 
     |████████████████████████████████| 139 kB 88.0 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |████████████████████████████████| 2.3 MB 64.0 MB/s 
     |████████████████████████████████| 378 kB 75.4 MB/s 
     |████████████████████████████████| 50 kB 7.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 13.7 MB/s 
     |████████████████████████████████| 7.6 MB 75.6 MB/s 
     |████████████████████████████████| 182 kB 93.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 12.5 MB/s 
     |████████████████████████████████| 132 kB 70.8 MB/s 
     |███████

In [ ]:
from transformers import BertTokenizer, BertModel,BertConfig
configuration = BertConfig(num_hidden_layers=2)
model = BertModel.from_pretrained("bert-base-uncased",config=configuration)


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['bert.encoder.layer.8.output.LayerNorm.bias', 'bert.encoder.layer.6.attention.self.query.weight', 'bert.encoder.layer.5.attention.self.query.weight', 'bert.encoder.layer.5.intermediate.dense.weight', 'bert.encoder.layer.11.attention.output.LayerNorm.weight', 'bert.encoder.layer.4.attention.self.value.weight', 'bert.encoder.layer.6.attention.output.LayerNorm.weight', 'bert.encoder.layer.7.attention.self.key.bias', 'bert.encoder.layer.2.attention.output.dense.bias', 'bert.encoder.layer.4.attention.self.value.bias', 'bert.encoder.layer.4.attention.output.LayerNorm.weight', 'bert.encoder.layer.7.attention.self.query.bias', 'bert.encoder.layer.8.attention.self.value.weight', 'bert.encoder.layer.5.output.dense.bias', 'bert.encoder.layer.7.attention.output.dense.weight', 'bert.encoder.layer.10.attention.output.dense.weight', 'bert.encoder.layer.10.attention.self.query.weight', 'bert.encoder.l

In [ ]:
import torch
import numpy as np
len_dataset=32
img_seq_len=10
tabular_seq_len=4
img_embeddings=torch.from_numpy(np.random.rand(len_dataset,img_seq_len,768)).float()
tabular_embeddings=torch.from_numpy(np.random.rand(len_dataset,tabular_seq_len,768)).float()
# radiomics_embeddings=torch.from_numpy(np.random.rand(1,3,768)).float()

In [ ]:
a = torch.empty(len_dataset).uniform_(0, 1)
y=torch.bernoulli(a)

In [ ]:
import numpy as np
batch_size=4

token_type_ids=torch.tensor(torch.cat([torch.zeros(batch_size,img_seq_len),torch.ones(batch_size,tabular_seq_len)],axis=-1),dtype=torch.int32)
position_ids=torch.tensor([list(range(img_seq_len))+list(range(tabular_seq_len)) for i in range(batch_size)],dtype=torch.int32)

<ipython-input-5-3fc0f8ea091e>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  token_type_ids=torch.tensor(torch.cat([torch.zeros(batch_size,img_seq_len),torch.ones(batch_size,tabular_seq_len)],axis=-1),dtype=torch.int32)


In [ ]:
import torch.nn as nn
class MyModel(nn.Module):
  def __init__(self,model,num_output_classes=2):
    super(MyModel,self).__init__()
    self.transformer_model=model
    self.linear1=nn.Linear(768,256)
    self.relu=nn.ReLU()
    self.linear2=nn.Linear(256,num_output_classes)
  def forward(self,random_embeddings,position_ids,token_type_ids):
    bert_output=model(inputs_embeds=random_embeddings,position_ids=position_ids,token_type_ids=token_type_ids)
    bert_output_avg=bert_output.last_hidden_state.mean(axis=1)
    x=self.linear1(bert_output_avg)
    x=self.relu(x)
    x=self.linear2(x)
    return x



In [ ]:
mymodel=MyModel(model,num_output_classes=2)

In [ ]:
from torch.utils.data import Dataset,DataLoader
# pass image embeddings and tabular data embeddings
class MyDataset(Dataset):
  def __init__(self,img_data,tabular_data,labels):
    self.img=img_data
    self.tab=tabular_data
    self.labels=labels
  def __len__(self):
    return len(self.img)
  def __getitem__(self,index):
    img_instance=self.img[index]
    tab_instance=self.tab[index]
    random_embedding=torch.cat([img_instance,tab_instance],axis=0)
    label_instance=self.labels[index]
    return random_embedding,label_instance

In [ ]:
mydataset=MyDataset(img_embeddings,tabular_embeddings,y)

In [ ]:
dataloader=DataLoader(mydataset,batch_size=batch_size)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mymodel.parameters(), lr=0.001)

In [ ]:
Epochs=1
for epoch in range(Epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        labels=labels.long()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = mymodel(inputs,position_ids,token_type_ids)
        loss = criterion(outputs, labels)
        print(loss)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
      

print('Finished Training')

tensor(0.7102, grad_fn=<NllLossBackward0>)
tensor(0.6249, grad_fn=<NllLossBackward0>)
tensor(0.7135, grad_fn=<NllLossBackward0>)
tensor(2.0896, grad_fn=<NllLossBackward0>)
tensor(0.4812, grad_fn=<NllLossBackward0>)
tensor(0.7790, grad_fn=<NllLossBackward0>)
tensor(1.1277, grad_fn=<NllLossBackward0>)
tensor(1.1892, grad_fn=<NllLossBackward0>)
Finished Training
